## General Comments

- The **Monte Carlo** methods of evaluating conditional mean for option pricing is **nearly always the method of choice for high-dimensional problems**. 
- It is also easy to apply to problems with **non-Markovian dynamics** as well as **strong path-dependency in the payout**, 
    - For **PDE or finite-difference method** to handle path-dependency and non-Markovian dynamics, the usual approach is to introduce auxiliary variables, so that in the higher-dimensional space, the problem is both Markovian and path-independent (an example is the formulation of lookback options). 
    - But the above reformulation can be non-trivial sometimes, and bringing extra dimensions also put a toll on runtime.
- However, it is **not straightforward to apply Monte Carlo on backward induction problems**, such as American or Bermudan option pricing.
    - One remedy is the infamous [Longstaff-Schwarz regression](https://www.evernote.com/shard/s191/nl/21353936/59374c6a-17ad-617f-2d78-9426eb9509b9?title=Longstaff%20and%20Schwartz%20(2001)), where the idea is as follows.
        - Simulate for a few paths, obtain some estimate of continuation value $y$ given the state variable values at any given intermediate points $X$.
        - Use regression or quadratic fit $y$ to $X$, and thus for any given $X$, one has an estimate of $y$.
        - Do full simulation, this time no need to do backward induction, as one has the estimate of the continuation value function above.
    - There seems quite some literature on this topic; see Glasserman's work.
- The drawback of Monte Carlo as compared to finite-difference or tree-based method, is that **Monte Carlo is usually slower**.
- The fundamental theoretical foundation of Monte Carlo is the **Strong Law of Large Numbers (LLN)** and **Central Limit Theorem (CLT)**.

## Generating Random Variables

### Inverse Transform

Let $Z\sim F^{-1}(U)$, then $Z$ has CDF $F$. One can generalize by using general inverse of $F$ rather than the true inverse.

### Acceptance-Rejection Method

For the target pdf $f(z)$, find a 'blanket' $e(z)$ and constant $C$ such that $C\cdot e(z)\geq f(z)$, and that $e(z)$ is easy to sample from.
- Draw a sample $Z$ from $e(z)$
- Draw an independent uniform variable $U$
- Accept the sample $Z$ if $U\leq f(Z)/(Ce(Z))$; otherwise reject it.

$C$ should be chosen as tight as possible so that it does not incur wasteful rejections.

### Composition

This is just a fancy name to use known mapping between random variables to generate from one to another, e.g. from Gaussian to log normal. Another example is to use linear transformation and cholesky decomposition to go from standardized multi-dimensional Gaussian to general multi-dimensional Gaussian.

## Generating Sample Paths for SDE - The Euler Scheme

Continuos dynamic of SDE: $dX_t = \mu(t, X_t)dt + \sigma(t, X_t)dW_t$

- **Explicit Scheme**: $\hat{X}_{i+1}=\hat{X}_{i}+\mu(i\Delta, \hat{X}_i)\Delta + \sigma(i\Delta, \hat{X}_i)(W(i\Delta+\Delta)-W(i\Delta))$.
- **Implicit Scheme**: $\hat{X}_{i+1}=\hat{X}_{i}+\mu(i\Delta, \hat{X}_{i+1})\Delta + \sigma(i\Delta, \hat{X}_i)(W(i\Delta+\Delta)-W(i\Delta))$

Some comments.
- The implicit scheme **only discretized the drift term implicitly, and not the diffusion term**. This is because the stochastic Ito integral is defined to be non-anticipative, in the sense that the integrand is always evaluated 'to the left' on any partitions of the Brownian motion.
- An approximation is **weakly consistent** if there exists a function $c(\Delta)$ with $c(\Delta)\downarrow 0$ as $\Delta\downarrow 0$, such that
\begin{align}
E\left(|E(\Delta^{-1}(\hat{X}_{i+1}-\hat{X}_{i})|\mathbb{F}_{i\Delta})-\mu(i\Delta, \hat{X}_i)|^2\right)\leq c(\Delta),
E\left(|E(\Delta^{-1}(\hat{X}_{i+1}-\hat{X}_{i})(\hat{X}_{i+1}-\hat{X}_{i})^{T}|\mathbb{F}_{i\Delta})-\sigma(i\Delta, \hat{X}_i)\sigma(i\Delta, \hat{X}_i)^{T}|^2\right)\leq c(\Delta).
\end{align}
An approximation is **weak convergence of order $\beta$** if 
\begin{align}
|E(g(X(T)))-E(g(\hat{X}(T)))|\leq c\Delta^\beta,
\end{align}
for all $\Delta\in(0, \Delta_0)$, where $\Delta_0$ and $c$ are constants and $c$ does not depend on $\Delta$ (but may depend on $g$). Both Euler schemes are weakly consistent and weak convergent of order 1, but the implicit scheme is numerically more stable.

Some practical concerns.
- **Linear-Drift SDE**. The **restricted stability region** of the Euler scheme can be a practical concern. 
    - For instance, the SDEs of the mean-reversion type
    \begin{align}
    dX(t)=\kappa(\theta(t)-X(t))dt + \sigma(t, X(t))dW(t).
    \end{align}
    **When $\kappa$ is big, the Euler scheme can become unstable and return meaningless results**. A way to get around that is to remove the drift by multiplying $X(t)$ with an exponential term, and then solve the driftless SDE.
- **Log-Euler Scheme**. This is to deal with the case where the stochastic process is **required to be nonnegative**. In those cases, just take logarithm and model the resulting log stochastic process.

## Sensitivity Estimates - Simulation for the Greeks

### Finite Difference Estimate

Let $V(\alpha)=E(Y(\alpha))$. Then we can use a finite difference of the Monte Carlo sum to estimate a derivative:
\begin{align}
\lim_{n\rightarrow\infty}\frac{\bar{Y}_n(\alpha+\epsilon)-\bar{Y}_n(\alpha-\epsilon)}{2\epsilon}
\end{align}
To minimize variance, one should use the same sample for $\bar{Y}_n(\alpha+\epsilon)$ or $\bar{Y}_n(\alpha-\epsilon)$, which is called the **common random number scheme**.

### Pathwise Estimate

In its most general case, it is also called the **infinitesimal perturbation analysis**. The idea is **differentiate the payoff function before expectation**.
\begin{align}
\frac{dV}{d\alpha} = \frac{d}{d\alpha}E(Y(\alpha))=E\left(\frac{d}{d\alpha}Y(\alpha)\right)=E\left(\frac{d}{d\alpha}g(X(\alpha))\right)
\end{align}

The right-hand side expectation can be estimated using Monte Carlo simulation. 

The exchange of integration (expectation) and derivatives above needs some regulatory conditions. In particular, if $g$ is not continuous, pathwise estimate will yield the wrong results. 
- For instance, if $g$ is a step function, $g'$ will be a Dirac function. By simulation, $g'(X(\alpha))$ will be zero everywhere, yielding a zero estimate for the Greek; but expectation of the Dirac function, i.e. the theoretical value of the Greek is certainly not zero.

Besides the restriction that pathwise estimate cannot apply to payoff functions that are not continuous, it can also be **cumbersome for multi-dimensional SDEs**, where there are several state variables, since a lot of differentiation need to be done.

### Likelihood Ratio Method


The idea is to **differentiate the density function, rather than the payoff function**, and make it into a likelihood ratio:
\begin{align}
\frac{\partial V(\alpha)}{\partial\alpha} = \int g(x)\frac{\partial f(x;\alpha)}{\partial\alpha}dx = \int g(x)\frac{\partial\log f(x;\alpha)}{\partial\alpha}f(x;\alpha)dx.
\end{align}

**Pros**

Compared to pathwise estimate method, the likelihood ratio method requires little knowledge of the payout function and its derivatives (and thus does not really care if it is continuous or not), making it convenient for general implementation on a computer. 

**Cons**

- As such, it is not applicable if only the final payoff function $g$ is a function of $\alpha$.

- A main obstacle of using the likelihood method is one need to know the explicit form of the transition density $f(x;\alpha)$, and that it is easy to differentiate it, though sometimes a finite difference procedure on top can help.

- Another annoyance of likelihood ratio method is that its variance can often be quite big, particularly if the parameter $\alpha$ simultaneously affects multiple stochastic variables.

- When both pathwise and likelihodd ratio methods apply, however, the pathwise method generally is more efficient.

### Final Comment

A final word about these sensitivity methods is that they are often used in conjunture. For instance, while it is common to use either the pathwise method or the likelihood ratio method to compute first order sensitivities (such as delta), second-order sensitivitities (such as gamma) are often done by the finite difference method applied to first-order sensitivities, often using fairly sizable shifts of the underlying variables.

## Variance Reduction Techniques

As the name suggests, the variance reduction technique seeks to reduce the variance of Monte Carlo sample sums.

### Antithetic Variates

Suppose we want to estimate the mean of a random variable $Y=H(U)$, where $H$ is a function and $U$ is a vector of independent uniformly distributed random variables. Let $U=(U_1, \dots, U_p)$ and $\tilde U=(1-U_1, \dots, 1-U_p)$. Then the **antithetic estimate** is 
\begin{align}
\frac{H(U)+H(\tilde U)}{2}.
\end{align}
The effect of reduced variance is most apparent if $H$ is monotonic in $U$.

The general idea of applying deterministic transformations to a vector-valued sample of random numbers as a way to reduce variance is sometimes known as **systematic sampling**.

### Control Variates

Suppose we want to estimate the mean of $E(Y)$, and let $Y^c$ be a control variate with closed-form mean $E(Y^c)=\mu^c$, ideally **strongly correlated (no need to be negatively)** with $Y$. Then instead of forming sample sums of $Y$, one can sample the following random variable:
\begin{align}
X= Y-\beta(Y^c-\mu^c).
\end{align}
Clearly the estimated mean does not change:
\begin{align}
E(X)=E(Y)-\beta(E(Y^c)-\mu^c)=E(Y).
\end{align}
And depending on the covariance between $Y$ and $Y^c$, we can choose some $\beta$ to minimize the overall variance of $X$.

### Importance Sampling

It is relating to change of measure in quant finance:
\begin{align}
\mu=E(g(X))=\int g(x)f(x)dx = \int g(x)\frac{f(x)}{h(x)}h(x)dx=\tilde E\left(g(X)\frac{f(X)}{h(X)}\right).
\end{align}
Delibrately choosing a density function $h$ (or equivalently, a distribution to sample from) where values of $g(X)$ are high (high 'importance') is shown to be a good rule of thumb to reduce variance.
- As an example, consider the toy problem of estimating $P(X>5)$, where $X$ is a standard Gaussian. As such, $g(X) = 1(X>5)$.
- Without importance sampling, the variance would be too high compared to the true value of the expectation/probability. The symptom of this is a lot of sample paths are needed in order to have a non-zero estimate of the probability.
- However, if we choose $h(x)\sim N(5, 1)$, one can avoid such cunnumdrum while maintaining an unbiased estimate.

## References

- [< Interest Rate Modeling >](https://www.evernote.com/shard/s191/nl/21353936/eafeaca1-2784-4f1b-9caf-6802485887c5?title=Interest_Rate_Modeling_by_Piterbarg.pdf), Chapter 3.